# 매매 분석·평가 메타 에이전트

기사·코인가격·공포/탐욕 전문가의 분석 결과와 사용자 매매내역·매매 의도를 종합해, 해당 매매에 대한 분석 및 평가를 제시합니다.

- **입력**: 평가 기간, 세 전문가 의견 요약, 매매 내역, 사용자 매매 의도(선택)
- **출력**: Pydantic 스키마 (기사/코인가격/공포탐욕 전문가 의견 각각 평가, 본인 에이전트의 매매 분석·평가, 제안)

## 1. 환경 설정

In [37]:
from dotenv import load_dotenv

load_dotenv()

python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 8
python-dotenv could not parse statement starting at line 9
python-dotenv could not parse statement starting at line 10
python-dotenv could not parse statement starting at line 11
python-dotenv could not parse statement starting at line 12
python-dotenv could not parse statement starting at line 13
python-dotenv could not parse statement starting at line 14
python-dotenv could not parse statement starting at line 15
python-dotenv could not parse statement starting at line 16
python-dotenv could not parse statement starting at line 17
python-dotenv could not parse statement starting at line 18
python-dotenv could not parse statement starting at line 19


True

In [38]:
# LangSmith 추적 (선택) https://smith.langchain.com
from langchain_teddynote import logging
logging.langsmith("trade-evaluation-expert-agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
trade-evaluation-expert-agent


In [39]:
import pandas as pd
from pathlib import Path

from langchain_core.prompts import load_prompt
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

## 2. 데이터 로드

CSV 경로는 상수로 두어 추후 API로 교체 시 `load_trade_history()`만 수정하면 됩니다.

In [40]:
def get_csv_path() -> Path:
    """노트북과 같은 디렉터리의 CSV 경로 (추후 API로 교체 가능)."""
    return Path("trade_history.csv")

def load_trade_history(csv_path: Path | None = None) -> pd.DataFrame:
    csv_path = csv_path or get_csv_path()
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()
    df["trade_time"] = pd.to_datetime(df["trade_time"])
    df = df.sort_values("trade_time").reset_index(drop=True)
    return df

df = load_trade_history()
df.head(10)

,trade_type,price,quantity,total_price,fee,trade_time,created_at,profit_loss_rate,avg_buy_price
0,0,9.51,5257.623554,50000.000000,25.000000,2021-05-08 04:54:13,2025-07-25 14:19:37.703151,NaN,NaN
1,0,748.00,66.778108,49950.024986,24.975012,2021-05-08 04:59:04,2025-07-25 14:19:37.703151,NaN,NaN
2,1,9.37,5257.623554,49263.932702,24.631966,2021-05-08 15:54:40,2025-07-25 14:19:37.703151,-1.47,9.510000
3,1,834.00,66.778108,55692.942297,27.846471,2021-05-08 16:10:11,2025-07-25 14:19:37.703151,11.50,748.000000
4,0,1595.00,65.532234,104523.913039,52.261957,2021-05-08 16:21:23,2025-07-25 14:19:37.703151,NaN,NaN
5,0,1585.00,63.266789,100277.861056,50.138931,2021-05-08 16:28:24,2025-07-25 14:19:37.703151,NaN,NaN
6,1,1000.00,128.799023,128799.023190,64.399512,2021-05-19 21:51:09,2025-07-25 14:19:37.703151,-37.11,1590.087945
7,0,11100.00,11.591862,128669.665092,64.334833,2021-05-19 22:41:08,2025-07-25 14:19:37.703151,NaN,NaN
8,1,11500.00,11.591862,133306.409780,66.653205,2021-05-19 22:46:38,2025-07-25 14:19:37.703151,3.60,11100.000000
9,0,1620.00,82.205811,133173.413285,66.586707,2021-05-20 01:40:22,2025-07-25 14:19:37.703151,NaN,NaN


In [41]:
def get_trades_for_evaluation(
    df: pd.DataFrame,
    target_date_start: str,
    target_date_end: str,
    max_trades: int | None = 100,
) -> str:
    """평가 대상 기간의 매매 내역을 포맷된 문자열로 반환."""
    start = pd.to_datetime(target_date_start).normalize()
    end = pd.to_datetime(target_date_end).normalize() + pd.Timedelta(days=1)
    mask = (df["trade_time"] >= start) & (df["trade_time"] < end)
    block = df.loc[mask].copy()
    if max_trades is not None:
        block = block.tail(max_trades)
    lines = []
    for _, r in block.iterrows():
        side = "매도" if r["trade_type"] == 1 else "매수"
        ts = r["trade_time"].strftime("%Y-%m-%d %H:%M:%S")
        row_str = f"{ts} | {side} | {r['price']} | {r['quantity']} | {r['total_price']} | 수수료 {r['fee']}"
        if r["trade_type"] == 1 and pd.notna(r.get("profit_loss_rate")):
            row_str += f" | 손익률 {r['profit_loss_rate']}%"
            if pd.notna(r.get("avg_buy_price")):
                row_str += f" | 평균매수가 {r['avg_buy_price']}"
        lines.append(row_str)
    return "\n".join(lines) if lines else "(해당 기간 매매 없음)"

# 예시
get_trades_for_evaluation(df, "2022-01-12", "2022-01-14")[:1200]

'2022-01-12 18:52:23 | 매수 | 932.0 | 107.29613734 | 100000.00000088 | 수수료 50.0\n2022-01-12 18:58:07 | 매수 | 9815.0 | 10.17829852 | 99899.9999738 | 수수료 49.94999999\n2022-01-12 19:01:12 | 매도 | 9840.52175358 | 10.17829852 | 100159.7680005 | 수수료 50.079884 | 손익률 0.26% | 평균매수가 9815.0\n2022-01-12 19:18:07 | 매수 | 9760.0 | 10.25204918 | 100059.9999968 | 수수료 50.03\n2022-01-12 19:18:31 | 매도 | 9690.0 | 10.25204918 | 99342.3565542 | 수수료 49.67117828 | 손익률 -0.72% | 평균매수가 9760.0\n2022-01-12 19:25:10 | 매수 | 9520.0 | 10.37015452 | 98723.8710304 | 수수료 49.36193552\n2022-01-12 19:33:41 | 매도 | 9480.0 | 10.37015452 | 98309.0648496 | 수수료 49.15453242 | 손익률 -0.42% | 평균매수가 9520.0\n2022-01-12 20:08:38 | 매도 | 935.0 | 107.29613734 | 100321.8884129 | 수수료 50.16094421 | 손익률 0.32% | 평균매수가 932.0\n2022-01-12 21:30:07 | 매수 | 9140.0 | 21.76706783 | 198950.9999662 | 수수료 99.47549998\n2022-01-12 21:35:50 | 매도 | 9175.0 | 21.76706783 | 199712.84734025 | 수수료 99.85642367 | 손익률 0.38% | 평균매수가 9140.0\n2022-01-13 13:55:45 | 매수 | 9110.0

## 3. Pydantic 응답 스키마 및 Parser

In [42]:
class TradeEvaluationExpertResponse(BaseModel):
    article_expert_evaluation: str = Field(
        description="기사 전문가 의견에 대한 평가 (해당 의견의 타당성, 매매와의 관계 등)"
    )
    coin_price_expert_evaluation: str = Field(
        description="코인가격 전문가 의견에 대한 평가 (해당 의견의 타당성, 매매와의 관계 등)"
    )
    fear_greed_expert_evaluation: str = Field(
        description="공포/탐욕 전문가 의견에 대한 평가 (해당 의견의 타당성, 매매와의 관계 등)"
    )
    own_trade_analysis: str = Field(
        description="본인(이 에이전트)의 유저 매매에 대한 분석 및 평가 (타이밍, 손익, 의도 반영 여부 등)"
    )
    suggestions: str = Field(
        description="다음 매매나 습관에 대한 구체적 제안/권고"
    )

parser = PydanticOutputParser(pydantic_object=TradeEvaluationExpertResponse)

## 4. 프롬프트, LLM, 체인

In [43]:
prompt = load_prompt("prompts/trade_evaluation_expert.yaml", encoding="utf-8")
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

In [44]:
MODEL_NAME = "gpt-4.1"
llm = ChatOpenAI(temperature=0, model=MODEL_NAME)
chain = prompt | llm | parser

## 5. 평가 기간·전문가 의견·매매 내역 입력 및 실행

In [45]:
# 2019-02-19 기준 BTC 임시 거래 1건 (하드코딩)
target_date_start = "2019-02-19"
target_date_end = "2019-02-19"
target_period = "2019-02-19"
trade_history_text = """2019-02-19 12:00:00 | 매도 | 3920.00 | 0.05 | 196.00 | 수수료 0.098 | 손익률 1.03% | 평균매수가 3880.00"""

In [46]:
# 세 전문가 의견 요약 (직접 입력 또는 추후 article/coin_price/fear_greed 체인 호출로 채움)
expert_article_summary = """ArticleExpertResponse(verdict='긍정적', market_flow_analysis="2019년 2월 19일 전후로 암호화폐 시장은 뚜렷한 상승 전환 신호를 보이고 있습니다. 해당일 헤드라인에서는 비트코인이 4000달러 돌파를 시도하며, 이더리움, 이오스 등 주요 알트코인도 급등세를 기록하고 있습니다. 거래량 역시 9개월 만에 최고치를 기록하며 시장에 강한 매수세가 유입되고 있음을 보여줍니다. 이더리움 하드포크 기대감, 비트메인 신제품 공개, 기관투자 장기투자 언급 등 긍정적 이슈가 다수 포착됩니다. 전반적으로 '암호화폐 해빙기'라는 표현이 등장할 정도로 투자심리가 개선되고 있으며, 이전까지의 약세·횡보 국면에서 벗어나 단기 랠리 분위기가 조성되고 있습니다.", short_long_term_perspective='단기적으로는 이더리움 하드포크, 비트코인 4000달러 돌파 시도, 주요 알트코인 급등 등 강한 매수세와 기대감이 시장을 주도하고 있습니다. 거래량 증가와 함께 투자심리도 뚜렷이 개선되었습니다. 장기적으로 보면, 2월 초부터 이어진 횡보 및 약세장이 점차 완화되고, 기관투자 유입 기대, 블록체인 실사용 확대, 글로벌 규제 논의 등 긍정적 구조 변화가 감지됩니다. 다만, 일부 부정적 이슈(비트메인 실적 악화, 블록체인 협회 잡음 등)도 존재하지만, 전체적으로는 상승 전환의 신호가 더 강하게 나타납니다.', notable_periods=[NotablePeriod(title='비트코인 4000달러 돌파 시도 및 시장 급등', period_text='2019-02-13 ~ 2019-02-19', period_data=[{'2019-02-13': '비트코인 3650선 하회, 약보합세. 장 후반 반등 시도'}, {'2019-02-14': '비트코인 3600달러 돌파 시험, 전반적 약세 분위기'}, {'2019-02-15': 'JP모간 자체 암호화폐 개발, 시장은 약세 지속'}, {'2019-02-16': '기관투자 증가, 비트코인 3600선 횡보'}, {'2019-02-17': '비트코인 3600 박스권, 일부 알트코인 급등'}, {'2019-02-18': '비트코인 3600선에서 상승세, 이더리움 8% 급등'}, {'2019-02-19': '비트코인 4000달러 돌파 시도, 이더리움·이오스 등 급등, 거래량 9개월 최고'}]), NotablePeriod(title='이더리움 하드포크 기대감 및 알트코인 강세', period_text='2019-02-16 ~ 2019-02-19', period_data=[{'2019-02-16': '온톨로지 등 일부 알트코인 급등, 시장 혼조'}, {'2019-02-17': '이더리움, 온톨로지 등 알트코인 강세'}, {'2019-02-18': '이더리움 8% 급등, 하드포크 기대감 부각'}, {'2019-02-19': '이더리움 하드포크 앞두고 급등, 선물 거래 주목'}]), NotablePeriod(title='기관투자 및 블록체인 실사용 확대 기대', period_text='2019-02-13 ~ 2019-02-19', period_data=[{'2019-02-13': '모간 크릭, 공적 연금 투자 유치 성공'}, {'2019-02-14': '기관 자금 6~12개월 내 유입 전망'}, {'2019-02-15': 'JP모간, 자체 암호화폐 개발 발표'}, {'2019-02-19': '전문 투자회사, 기관의 장기 투자 언급'}])])"""
expert_coin_price_summary = "FearGreedExpertResponse(verdict='부정적', market_flow_analysis='2019-02-19의 공포/탐욕 지수는 65로, 최근 며칠 사이 급격한 상승세를 보이며 극단적 탐욕 구간에 진입했습니다. 2월 17일 38에서 2월 18일 63, 2월 19일 65로 단기간에 27포인트 이상 급등하였으며, 이는 투자자 심리가 과열되어 있음을 시사합니다. 과거 6개월간 지수 흐름을 보면 60대 중반까지 오른 적이 거의 없으며, 이처럼 단기간 급등은 단기 조정 가능성을 높입니다.', short_long_term_perspective='단기: 최근 3일간 지수가 38→63→65로 급등해 단기적으로 과열 신호가 매우 강합니다. 이 구간에서는 추가 상승보다는 차익 실현 매도세가 유입될 가능성이 높으므로 매수에는 부정적입니다. 장기: 6개월 구간에서 65는 최상단에 해당하며, 과거에도 50대 중후반 이상에서는 이후 조정이 자주 발생했습니다. 장기적으로도 추가 상승 여력보다는 단기 과열 해소 구간 진입 가능성이 높아 보입니다.', notable_periods=[NotablePeriod(title='극단적 탐욕으로의 급격한 전환', period_text='2019-02-17 ~ 2019-02-19', period_data=[{'2019-02-17': 38}, {'2019-02-18': 63}, {'2019-02-19': 65}]), NotablePeriod(title='2019년 1월 초 단기 급등', period_text='2019-01-02 ~ 2019-01-04', period_data=[{'2019-01-02': 30}, {'2019-01-03': 33}, {'2019-01-04': 48}]), NotablePeriod(title='2018년 11월 중순~말 극단적 공포 구간', period_text='2018-11-21 ~ 2018-11-27', period_data=[{'2018-11-21': 15}, {'2018-11-22': 14}, {'2018-11-23': 12}, {'2018-11-24': 15}, {'2018-11-25': 9}, {'2018-11-26': 17}, {'2018-11-27': 11}]), NotablePeriod(title='2018년 9월 말~10월 초 탐욕 구간', period_text='2018-09-22 ~ 2018-09-28', period_data=[{'2018-09-22': 35}, {'2018-09-23': 38}, {'2018-09-24': 43}, {'2018-09-25': 37}, {'2018-09-26': 37}, {'2018-09-27': 42}, {'2018-09-28': 42}])])"
expert_fear_greed_summary = "CoinPriceExpertResponse(verdict='보통', market_flow_analysis='2019-02-19는 전일(2/18) 4.85%의 강한 상승 이후, 추가 상승(고가 4338000) 시도가 있었으나 종가는 4259000으로 소폭 상승(+0.54%)에 그쳤습니다. 2/08 이후 단기 급등(5.97%)과 2/18의 강한 상승이 연속적으로 나타났으나, 2/19에는 변동성이 다소 줄고 윗고리가 긴 형태로 마감되어 단기 과열 및 매도세 유입이 감지됩니다. 전반적으로 1월 말~2월 초까지는 횡보 및 약세였으나, 2월 중순 이후 단기 반등세가 뚜렷하게 나타난 구간입니다.', short_long_term_perspective='단기: 2/08 이후 강한 반등세가 이어졌고, 2/18에 대량 상승이 발생한 직후라 단기적으로는 과열 신호와 차익실현 매물 출회 가능성이 높아 보입니다. 2/19 당일은 추가 상승 시도 후 윗고리가 길게 마감되어 단기 매수 진입은 신중해야 할 시점입니다. 장기: 2018년 하반기부터 이어진 하락 추세에서 12월~1월 저점(360만~380만 원대) 이후 바닥권에서 반등이 시도되고 있으나, 전체적으로는 아직 명확한 추세 전환 신호가 부족합니다. 장기적으론 저점 대비 반등 구간이나, 본격적인 상승 전환으로 보기엔 이른 시점입니다.', notable_periods=[NotablePeriod(title='2월 중순 단기 급등 구간', period_text='2019-02-08 ~ 2019-02-19', period_data=[{'2019-02-08': '3783000|4075000|3768000|4009000, +5.97%'}, {'2019-02-09': '4007000|4038000|3968000|3998000, -0.27%'}, {'2019-02-10': '3997000|4030000|3937000|4023000, +0.63%'}, {'2019-02-17': '4002000|4094000|3986000|4040000, +0.95%'}, {'2019-02-18': '4040000|4260000|4027000|4236000, +4.85%'}, {'2019-02-19': '4236000|4338000|4150000|4259000, +0.54%'}]), NotablePeriod(title='2018년 11월 중순~12월 초 급락 구간', period_text='2018-11-14 ~ 2018-11-20', period_data=[{'2018-11-14': '7248000|7270000|6514000|6782000, -6.43%'}, {'2018-11-15': '6782000|6811000|6253000|6529000, -3.73%'}, {'2018-11-16': '6529000|6580000|6350000|6425000, -1.59%'}, {'2018-11-19': '6431000|6436000|5563000|5618000, -12.64%'}, {'2018-11-20': '5618000|5762000|4935000|5177000, -7.85%'}]), NotablePeriod(title='2018년 12월 중순 저점 형성 및 반등 시도', period_text='2018-12-13 ~ 2018-12-18', period_data=[{'2018-12-13': '3889000|3900000|3693000|3721000, -4.30%'}, {'2018-12-14': '3721000|3777000|3595000|3642000, -2.12%'}, {'2018-12-15': '3641000|3685000|3562000|3619000, -0.63%'}, {'2018-12-16': '3620000|3700000|3612000|3637000, +0.50%'}, {'2018-12-17': '3636000|3973000|3625000|3923000, +7.86%'}, {'2018-12-18': '3921000|4110000|3848000|4088000, +4.21%'}]), NotablePeriod(title='2018년 9월 초 급락 구간', period_text='2018-09-05 ~ 2018-09-08', period_data=[{'2018-09-05': '8271000|8333000|7687000|7699000, -6.92%'}, {'2018-09-06': '7698000|7718000|7264000|7523000, -2.29%'}, {'2018-09-07': '7522000|7543000|7292000|7382000, -1.87%'}, {'2018-09-08': '7391000|7439000|7110000|7206000, -2.38%'}])])"
user_intent = "의미없는 공포감에 진행했다. 잘한 선택인지는 두고봐야 알겠지"

In [47]:
response = chain.invoke({
    "target_period": target_period,
    "expert_article_summary": expert_article_summary,
    "expert_coin_price_summary": expert_coin_price_summary,
    "expert_fear_greed_summary": expert_fear_greed_summary,
    "user_intent": user_intent,
    "trade_history_text": trade_history_text,
})
response

TradeEvaluationExpertResponse(article_expert_evaluation='기사 전문가 의견은 2019년 2월 19일 전후로 시장에 긍정적 모멘텀이 집중되었음을 잘 짚고 있습니다. 실제로 비트코인 4000달러 돌파 시도, 이더리움 하드포크 기대감, 거래량 급증 등은 단기적으로 투자심리 개선과 매수세 유입을 설명하는 데 타당합니다. 다만, 해당 의견은 주로 단기 랠리와 심리 개선에 초점을 두고 있어, 단기 과열이나 조정 가능성에 대한 경계는 상대적으로 약하게 언급되었습니다. 사용자의 매도 시점(2/19)과 비교하면, 기사 전문가의 긍정적 시각과는 달리 실제로는 단기 고점 신호가 일부 포착된 시점이기도 하므로, 매매와의 직접적 연관성은 제한적입니다.', coin_price_expert_evaluation='코인가격 전문가 의견은 가격 흐름과 캔들 패턴, 변동성, 단기 과열 신호를 균형 있게 분석하고 있습니다. 2/19 당일 윗고리가 긴 캔들, 전일 급등 이후 소폭 상승, 단기 매도세 유입 등은 실제 시장 상황과 부합합니다. 단기적으로는 추가 상승보다는 조정 가능성이 높다는 분석이 타당하며, 사용자의 매도 타이밍(2/19)과도 잘 맞아떨어집니다. 장기적으로는 아직 본격적 상승 전환 신호가 부족하다는 점도 현실적입니다.', fear_greed_expert_evaluation='공포/탐욕 전문가 의견은 공포/탐욕 지수의 급등을 근거로 단기 과열과 조정 가능성을 강조합니다. 3일 만에 38→65로 급등한 점, 과거 6개월간 유사 구간에서 조정이 잦았다는 점 등은 신뢰할 만한 근거입니다. 이 분석은 사용자의 매도 시점이 단기 과열 구간에 진입한 시점임을 뒷받침하며, 매매와의 연관성이 높고 타당성이 높다고 평가할 수 있습니다.', own_trade_analysis="사용자는 '의미없는 공포감'에 의해 2/19 12:00에 1.03%의 소폭 수익으로 매도하였습니다. 시장은 단기 급등 후 과열 신호가 강하게 나타난 시점이었고, 실제로 윗고리가 긴 캔들

In [ ]:
print("=== 기사 전문가 의견에 대한 평가 ===")
print(response.article_expert_evaluation)
print("\n=== 코인가격 전문가 의견에 대한 평가 ===")
print(response.coin_price_expert_evaluation)
print("\n=== 공포/탐욕 전문가 의견에 대한 평가 ===")
print(response.fear_greed_expert_evaluation)
print("\n=== 유저 매매 분석·평가 ===")
print(response.own_trade_analysis)
print("\n=== 제안 ===")
print(response.suggestions)

=== 기사 전문가 의견에 대한 평가 ===
기사 전문가 의견은 2019년 2월 19일 전후로 시장에 긍정적 모멘텀이 집중되었음을 잘 짚고 있습니다. 실제로 비트코인 4000달러 돌파 시도, 이더리움 하드포크 기대감, 거래량 급증 등은 단기적으로 투자심리 개선과 매수세 유입을 설명하는 데 타당합니다. 다만, 해당 의견은 주로 단기 랠리와 심리 개선에 초점을 두고 있어, 단기 과열이나 조정 가능성에 대한 경계는 상대적으로 약하게 언급되었습니다. 사용자의 매도 시점(2/19)과 비교하면, 기사 전문가의 긍정적 시각과는 달리 실제로는 단기 고점 신호가 일부 포착된 시점이기도 하므로, 매매와의 직접적 연관성은 제한적입니다.

=== 코인가격 전문가 의견에 대한 평가 ===
코인가격 전문가 의견은 가격 흐름과 캔들 패턴, 변동성, 단기 과열 신호를 균형 있게 분석하고 있습니다. 2/19 당일 윗고리가 긴 캔들, 전일 급등 이후 소폭 상승, 단기 매도세 유입 등은 실제 시장 상황과 부합합니다. 단기적으로는 추가 상승보다는 조정 가능성이 높다는 분석이 타당하며, 사용자의 매도 타이밍(2/19)과도 잘 맞아떨어집니다. 장기적으로는 아직 본격적 상승 전환 신호가 부족하다는 점도 현실적입니다.

=== 공포/탐욕 전문가 의견에 대한 평가 ===
공포/탐욕 전문가 의견은 공포/탐욕 지수의 급등을 근거로 단기 과열과 조정 가능성을 강조합니다. 3일 만에 38→65로 급등한 점, 과거 6개월간 유사 구간에서 조정이 잦았다는 점 등은 신뢰할 만한 근거입니다. 이 분석은 사용자의 매도 시점이 단기 과열 구간에 진입한 시점임을 뒷받침하며, 매매와의 연관성이 높고 타당성이 높다고 평가할 수 있습니다.

=== 본인(에이전트)의 유저 매매 분석·평가 ===
사용자는 '의미없는 공포감'에 의해 2/19 12:00에 1.03%의 소폭 수익으로 매도하였습니다. 시장은 단기 급등 후 과열 신호가 강하게 나타난 시점이었고, 실제로 윗고리가 긴 캔들, 공포/탐욕 지수 급등, 차익실현 매물 유입 등 단기